# 02 Vasking av data
Vi importerer et offentlig datasett og ser på vanlige vaskemidler

In [2]:
import pandas as pd

Vi bruker her oversikten over lusetellinger fra fiskehelse.no/ Mattilsynet igjen

In [9]:
df = pd.read_excel('data/fiskehelse_2017-10-27-lakselus_per_fisk.xlsx')

### Fikk vi med alle rader og kolonner?
Ta en kikk i Excel-filen din og se etter antall rader og antall kolonner og sammenlikn med tallet du med bruk av `df.shape`. Det første tallet er rader, det andre er kolonner

In [11]:
df.shape

(61757, 16)

### Hvilke kolonner har vi og hva slags datatype har de?

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61757 entries, 0 to 61756
Data columns (total 16 columns):
Uke                         61757 non-null int64
År                          61757 non-null int64
Lokalitetsnummer            61757 non-null int64
Lokalitetsnavn              45683 non-null object
Voksne hunnlus              22190 non-null float64
Lus i bevegelige stadier    22190 non-null float64
Fastsittende lus            22190 non-null float64
Brakklagt                   61757 non-null object
Kommune                     45683 non-null object
Fylkesnummer                45683 non-null float64
Fylke                       45683 non-null object
Lat                         45683 non-null float64
Lon                         45683 non-null float64
Lusegrense uke              61757 non-null object
Over lusegrense uke         38254 non-null object
Sjøtemperatur               23602 non-null float64
dtypes: float64(7), int64(3), object(6)
memory usage: 7.5+ MB


**Forklaring**: `int64` betyr heltall, `object` betyr som regel at det er tekst, `float64` betyr et tall med desimaler. 

### Fjern kolonner du ikke trenger
Gjør det det mer oversiktlig å jobbe med. Her kan vi fjerne `lat` og `lon` kolonnene som angir kartreferanse.

In [18]:
df = df.drop(['Lat', 'Lon'], axis=1)

**Forklaring**: Her lager vi en ny DataFrame med samme navnet, der vi dropper kolonnene Lat og Lon. 
`axis=1` betyr at det er kolonner vi skal droppe, 0 betyr rader.